<a id='home'></a>
### purpose

- create R scripts for training and fitting gradient forest models
- test them on a very small snps file

### outline

[1. Training script](#train)

[2. Fitting script](#fit)

[3. test training script](#test)

[4. test fitting script](#testfit)

In [1]:
from pythonimports import *

DIR = '/data/projects/pool_seq/phenotypic_data/offset_misc_files'

lview,dview = get_client()

latest_commit()
sinfo(html=True)

56 56
##################################################################
Today:	June 09, 2023 - 12:38:02
python version: 3.8.5
conda env: newpy385

Current commit of pythonimports:
commit 03d76f7a992130f4b94ac34a09ad439e918d3044  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Fri Jun 9 09:42:21 2023 -0400
##################################################################



<a id='train'></a>
# Training script

[top](#home)

In [2]:
training_script = '''
#--------------------------------------------------------------------------------------------------------#
# Given a set of populations, allele freqs, and environmental data, train gradient forests.
#
# Usage
# -----
# Rscript gradient_training.R snpfile envfile range_file basename save_dir imports_path
#
# Notes
# -----
# - snpfile and envfile must already be subset for training populations
#
# Parameters
# ----------
# snpfile - SNPs that you want to use to train gradient forests (row.names=pop_ID, cols=loci)
# envfile - lat/long, environmental data for each pop
# range_file - tab delimited .txt file path with columns for `lat`, `lon`, and each env used in training
#    - this will be used to interpolate model on pops' lat/long to a larger area
#      - specifically this should the range-wide environmental data that is from same 
#        timeframe as in `envfile` (eg 1961-1990)
#    - generally, these are the envdata extracted from ClimateNA GIS data, and then clipped
#        to the range map
#    - these can be current, future, or uniform (eg for common garden) climate
#    - see eg yeaman03:/notebooks/007_JP_gea/07_gradient_forest/01_prep_spatial_envdata.ipynb
# basename - the basename prefix wanted for saving files in `save_dir`
# save_dir - the directory where files should be saved
# imports_path - path to directory with my homebrew R functions
#--------------------------------------------------------------------------------------------------------#

library(data.table)
library(hash)
library(gradientForest)
library(raster)
library(rgeos)

options(datatable.fread.datatable=FALSE)

args = commandArgs(trailingOnly=TRUE)
snpfile <- args[1]
envfile <- args[2]
range_file <- args[3]
basename <- args[4]
save_dir <- args[5]
imports_path <- args[6]

source(paste0(imports_path, '/imports.R'))  # https://github.com/brandonlind/r_imports

print(format(Sys.time(), format="%B %d %Y %T"))

prep_snps <- function(snpfile, exclude=c(), sep='\\t'){
    #----------------------------------------------------------------#
    # READ IN FREQUENCYxPOP TABLE, REMOVE POPS IN exclude
    #
    # Notes
    # -----
    # - assumes snpfile has colname 'index' which has pop names
    # - the rest of the colnames are names for each locus in dataset
    #----------------------------------------------------------------#
    # load data
    snps <- fread(snpfile, sep=sep)
    # set row names
    row.names(snps) <- snps[ ,'index']
    # remove column so all cols are loci
    snps <- subset(snps, select = -c(index))
    # get popnames, exclude irrelevant columns
    pops <- rownames(snps)[!rownames(snps) %in% c('CHROM', 'POS')]
    if (len(exclude) > 0){
        # exclude any other pops
        pops <- pops[!pops %in% exclude]
    }
    # reframe datatable
    snps <- snps[pops, ]
    return(snps)
}

prep_envdata <- function(envfile, pops){
    #-------------------------------------------------------------------------#
    # COLOR GEOGRAPHIC GROUPS AND REORDER POPS IN ENVIRONMENT TABLE 
    #
    # Notes
    # -----
    # - places `envs` (names of environments + elevation) in global namespace
    #-------------------------------------------------------------------------#
    
    # read in the environmental data (previously centered/standardized)
    envdata <- read.csv(envfile, sep='\\t', row.names=1)

    envs <- c('AHM', 'CMD', 'DD5', 'DD_0', 'EMT', 'EXT', 'Eref', 'FFP', 'MAP', 'MAT', 'MCMT', 'MSP', 'MWMT',
              'NFFD', 'PAS', 'SHM', 'TD', 'bFFP', 'eFFP')

    stopifnot(all(envs %in% colnames(envdata)))
    cat(sprintf('There are %s environmental attributes in the envdata table.', len(envs)))
    
    # order pops same as snp table
    envdata <- envdata[pops, ]
    
    # add to global namespace so I don't have to return a stupid list, stupid R
    envs <<- envs
    
    return(envdata[, envs])
}




### SET UP NAMESPACE

# get allele freqs for each pop
cat(sprintf('\nPrepping snp data ...\n'))
cand_freqs <- prep_snps(snpfile) #, exclude=test_pops)

# get population names
pops <- rownames(cand_freqs)
cat(sprintf('\nPops ...\n'))
cat(sprintf(pops))

# prep training data (climate data at location of populations)
cat(sprintf('\n\nPrepping envdata\n ...'))
envdata <- prep_envdata(envfile, pops)
range_data <- read.csv(range_file, sep='\t')




### TRAIN GRADIENT FORESTS

cat(sprintf('\n\nTraining gradient forests ...\n'))

# double check that pops are in same order
print(rownames(envdata))
print(rownames(cand_freqs))
stopifnot(all(rownames(envdata)==rownames(cand_freqs)))

# NOTE: envdata below must have been filtered for env-only columns
# calc of maxLevel is different but not effectively from Ellis 2012 Supp 1 https://doi.org/10.6084/m9.figshare.c.3304350.v1
# recommended by Stephen Keller
maxLevel <- floor(log2(0.368*nrow(envdata)/2))
ptm <- proc.time()
gfOut <- gradientForest(cbind(envdata, cand_freqs),
                        predictor.vars=colnames(envdata[ ,envs]),
                        response.vars=colnames(cand_freqs),
                        ntree=500,
                        transform=NULL,
                        nbin=201,  # this isn't doing anything since I didn't use `compact`
                        maxLevel=maxLevel,
                        corr.threshold=0.5)
print(proc.time() - ptm)




### INTEROPOLATE MODEL TO RANGE_WIDE DATA FOR THE SAME TIME PERIOD
cat(sprintf('\n\nInterpolating gradient forests model ...\n'))
ptm <- proc.time()
predOut.gf <- predict(gfOut, range_data[ ,envs])
print(proc.time() - ptm)



### SAVE
cat(sprintf('\n\nSaving files ...\n'))
gfOut_trainingfile <- paste0(
    save_dir,
    sprintf('/%s_gradient_forest_training.RDS', basename)
)
saveRDS(gfOut, gfOut_trainingfile)

predfile <- paste0(
    save_dir,
    sprintf('/%s_gradient_forest_predOut.RDS', basename)
)
saveRDS(predOut.gf, predfile)

print(gfOut_trainingfile)
print(predfile)


### END
cat(sprintf('\nDONE!\n'))
print(format(Sys.time(), format="%B %d %Y %T"))

'''

In [3]:
script_file = op.join(DIR, 'gradient_training.R')

with open(script_file, 'w') as o:
    o.write(training_script)

!md5sum $script_file
# 8c1f41175f0d137fe1983ab82c3bacea - before revision1 md5sum

5d5099302b049d799dac88cc88e5974e  /data/projects/pool_seq/phenotypic_data/offset_misc_files/gradient_training.R


<a id='fit'></a>
# 2. fitting script 

[top](#home)

In [4]:
fitting_script = '''
#----------------------------------------------------------------------------------------------------------#
# Given a trained gradient forest, fit model to input climate data, `range_data`; save offset netCDF file.
#
# Usage
# -----
# Rscript gradient_fitting_script.R gfOut_trainingfile range_file predfile maskfile basename save_dir
#
# Parameters
# ----------
# gfOut_trainingfile - file path to trained gradient forest object, saved as .RDS
# range_file - tab delimited .txt file path with columns for `lat`, `lon`, and each env used in training
#    - generally, these are the envdata extracted from ClimateNA GIS data, and then clipped
#        to the range map
#    - these can be current, future, or uniform (eg for common garden) climate
#    - see eg yeaman03:/notebooks/007_JP_gea/07_gradient_forest/01_prep_spatial_envdata.ipynb
# predfile - path to RDS of object saved from interpolating gradient forest model across landscape
#    for the same time period as the data that went into training (created in gradient_training.R)
# maskfile - path to RDS object of a blank range map (same map used to clip range_file data)
#    - this was created by setting all values to 0, and then saving RDS
#    - see https://www.mountainmanmaier.com/software/pop_genom/
# basename - the basename prefix wanted for saving files in `save_dir`
# save_dir - the directory where files should be saved
#----------------------------------------------------------------------------------------------------------#

library(gradientForest)
library(raster)

args = commandArgs(trailingOnly=TRUE)
gfOut_trainingfile <- args[1]
range_file <- args[2]
predfile <- args[3]
maskfile <- args[4]
basename <- args[5]
save_dir <- args[6]

calc_euclid <- function(proj, curr){
    #-------------------------------------------------------------------#
    # CALCULATE EUCLIDEAN DISTANCE BETWEEN CURRENT AND PROJECTED OFFSET
    #-------------------------------------------------------------------#
    
    df <- data.frame(matrix(ncol=ncol(proj), nrow=nrow(proj)))
    stopifnot(all(colnames(proj) == colnames(curr)))
    for (i in 1:ncol(proj)){
        # square each difference between the column data
        df[,i] <- (proj[,i] - curr[,i])**2
    }
    # sum by rows then take the square root of each
    return (sqrt(apply(df, 1, sum)))
}

get_offset <- function(outfile){
    #--------------------------#
    # CALCULATE GENETIC OFFSET
    #--------------------------#
    
    # predict offset for common garden
    ptm <- proc.time()
    projOut <- predict(gfOut, range_data[, envs])
    print(proc.time() - ptm)

    # calc offset
    offset <- calc_euclid(projOut, predOut)

    # put offset into GIS layer
    m2 <- readRDS(maskfile)
    m2[range_cells] <- offset

    # save netCDF file
    writeRaster(m2,
                outfile,
                format='CDF',
                overwrite=TRUE)
    cat(sprintf('\nSaved netCDF file to:\n'))
    cat(sprintf(sprintf('\t%s', outfile)))
}



# SET UP NAMESPACE
envs <- c('Elevation', 'AHM', 'CMD', 'DD5', 'DD_0', 'EMT', 'EXT', 'Eref', 'FFP', 'MAP', 'MAT', 'MCMT', 'MSP', 'MWMT',
              'NFFD', 'PAS', 'SHM', 'TD', 'bFFP', 'eFFP')
stopifnot(length(envs) == 20)
gfOut <- readRDS(gfOut_trainingfile)
range_data <- read.csv(range_file, sep='\\t')
predOut <- readRDS(predfile)

# load mask shapefile, use to get cell IDs for each of the lat/long in range_data
mask <- readRDS(maskfile)
range_cells <- cellFromXY(mask, range_data[ ,c('lon','lat')])



# FIT GRADIENT FOREST MODEL TO INPUT ENVDATA
cat(sprintf('\nFitting gradient forest model to input envdata ...\n'))
ptm <- proc.time()
projOut <- predict(gfOut, range_data[ , envs])
print(proc.time() - ptm)



# CALCULATE OFFSET
cat(sprintf('\nCalculating offset ...\n'))
netCDF_file <- paste0(
    save_dir,
    sprintf('/%s_offset.nc', basename)
)
get_offset(netCDF_file)


# SAVE
outfile <- paste0(
    save_dir,
    sprintf('/%s_gradient_forest_fitted.RDS', basename)
)
saveRDS(projOut, outfile)

'''

In [5]:
fitting_file = op.join(DIR, 'gradient_fitting_script.R')

with open(fitting_file, 'w') as o:
    o.write(fitting_script)

!md5sum $fitting_file
# dc3f6d207850ad6730b539075f3a77be

dc3f6d207850ad6730b539075f3a77be  /data/projects/pool_seq/phenotypic_data/offset_misc_files/gradient_fitting_script.R


<a id='test'></a>
# 3. test training script

[top](#home)

In [21]:
snpfile = '/lu213/brandon.lind/data/testdir/snps_delete.txt'
# envfile = '/data/projects/pool_seq/environemental_data/change_p6/df_interior-naturalpops_raw_env-19variables_change-p6.txt'
envfile = '/data/projects/pool_seq/phenotypic_data/offset_misc_files/training/training_files/fdi-baypass-full_envdata.txt'
range_file = '/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_NORM_1961-1990/NA_NORM_1961-1990_all-envs_WGS84_clipped_interior.txt'
basename = 'test'
save_dir = '/lu213/brandon.lind/data/testdir'
imports_path = '/lu213/brandon.lind/mypy/r_imports'
Rscript = '/lu213/brandon.lind/data/anaconda3/envs/r35/bin/Rscript'  # r 3.5.1 env seen elsewhere in my notebooks

!$Rscript $script_file $snpfile $envfile $range_file $basename $save_dir $imports_path

hash-2.2.6.1 provided by Decision Patterns


Attaching package: ‘hash’

The following object is masked from ‘package:data.table’:

    copy

Loading required package: extendedForest
extendedForest 1.6.1
Type rfNews() to see new features/changes/bug fixes.
Loading required package: sp

Attaching package: ‘raster’

The following objects are masked from ‘package:hash’:

    values, values<-

The following object is masked from ‘package:data.table’:

    shift

rgeos version: 0.5-5, (SVN revision 640)
 GEOS runtime version: 3.8.0-CAPI-1.13.1 
 Linking to sp version: 1.4-5 
 Polygon checking: TRUE 

Warning message:
In fun(libname, pkgname) :
  rgeos: versions of GEOS runtime 3.8.0-CAPI-1.13.1
and GEOS at installation 3.4.2-CAPI-1.8.2differ
[1] "July 02 2021 10:27:58"

Prepping snp data ...

Pops ...
DF_p3 DF_p4 DF_p5 DF_p7 DF_p8 DF_p9 DF_p18 DF_p19 DF_p20 DF_p33 DF_p34 DF_p35 DF_p36 DF_p37 DF_p38 DF_p39 DF_p40 DF_p41 DF_p42 DF_p43 DF_p44 DF_p45 DF_p46 DF_p47 DF_p48 DF_p49 DF_p54 DF_p55 DF_

<a id='testfit'></a>

# 4. test fitting script

[top](#home)

In [25]:
gfOut_trainingfile = '/lu213/brandon.lind/data/testdir/test_gradient_forest_training.RDS'
future_range_file = '/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp45_2050s/NA_ENSEMBLE_rcp45_2050s_all-envs_WGS84_clipped_interior.txt'
predfile = '/lu213/brandon.lind/data/testdir/test_gradient_forest_predOut.RDS'
maskfile = '/data/projects/pool_seq/DF_datasets/DF_pooled_GEA/DF_pooled/snpsANDindels/interior_variety_without-p6/03_maf-p05_RD-recalculated_FDI/gradient_forest/mask.RDS'
basename = 'something'
save_dir = '/lu213/brandon.lind/data/testdir'

!$Rscript $fitting_file $gfOut_trainingfile $future_range_file $predfile $maskfile $basename $save_dir

Loading required package: extendedForest
extendedForest 1.6.1
Type rfNews() to see new features/changes/bug fixes.
Loading required package: sp

Fitting gradient forest model to input envdata ...
   user  system elapsed 
  0.228   0.036   0.264 

Calculating offset ...
   user  system elapsed 
  0.290   0.023   0.314 
Loading required namespace: ncdf4

Saved netCDF file to:
	/lu213/brandon.lind/data/testdir/something_offset.nc